# Fixed Income HW7
Cohort1 Group1 ‑ Ahmed Khwaja, Mingyue Qiu, Noah Sutherland, Vrushali Behara

In [1]:
import pandas as pd
import numpy as np
import warnings,math
from sympy import symbols,solve,exp,nsolve
import matplotlib.pyplot as plt
warnings.simplefilter(action='ignore', category=FutureWarning)

corr_mat,cho_mat,disc_func,vol_data=pd.read_excel('Homework_7_Data.xlsx',sheet_name=None,header=None).values()
disc_func.columns=['BT']
vol_data.columns=['vol']

4. Using the initial B(T) function given in the spreadsheet, solve for the 2-year, 3-year, 5-year, and 10-year par rates.

In [2]:
disc_func=disc_func.assign(T=lambda x:x.index/2+0.5,
                           par_rate=lambda x:((1/x['BT'])**(1/(2*x['T']))-1)*2)
disc_func[disc_func['T'].isin([2,3,5,10])][['T','par_rate']]

,T,par_rate
3,2.0,0.060321
5,3.0,0.062239
9,5.0,0.064301
19,10.0,0.067442


3. Following the methodology discussed in class, simulate the evolution of the vector of zero-coupon prices out to 10 years.

In [3]:
np.random.seed(719)
sim_df=pd.DataFrame(index=[np.linspace(0.5,10,20)])
for i in range(len(sim_df)):
    t=sim_df.index[i][0]
    if t==0.5:
        r=disc_func[disc_func['T']==0.5]['par_rate'].values[0]
        B_0_T=disc_func['BT'].values
    else:
        r=((1/sim_df.loc[t,t-0.5].values[0])**(1/(2*0.5))-1)*2
        B_0_T=sim_df[t-0.5].values[i:]
    sig_T_t=vol_data['vol'].values[:len(sim_df)-i]
    dz_i=np.random.multivariate_normal([0]*len(corr_mat),corr_mat)[:len(sim_df)-i]

    B_t_T=B_0_T+r*B_0_T*0.5+sig_T_t*B_0_T*dz_i
    sim_df[t]=[np.nan]*i+list(B_t_T)

sim_df

,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0,5.5,6.0,6.5,7.0,7.5,8.0,8.5,9.0,9.5,10.0
0.5,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1.0,0.968294,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1.5,0.937690,0.968863,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.0,0.903823,0.934368,0.964712,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.5,0.866926,0.896472,0.925776,0.952988,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,0.836802,0.864321,0.894138,0.912293,0.953654,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.5,0.805120,0.835968,0.866353,0.866948,0.902812,0.950352,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4.0,0.771059,0.803661,0.839223,0.822634,0.849842,0.898095,0.952348,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4.5,0.742947,0.767931,0.803066,0.780619,0.799958,0.853041,0.912830,0.960530,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.0,0.716874,0.739358,0.772775,0.732218,0.749790,0.803412,0.875722,0.923729,0.964361,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


5. Use the model to find the values of 2-year, 3-year, 5-year, and 10-year caps on Lt, where Lt denotes the six-month rate at time t implied by the six-month zero-coupon bond price B(t, t+0.50). Set the strike rate of each cap to the par rate identified in the previous question. For simplicity, assume that caplets are semiannual rather than quarterly. Recall that the first caplet is omitted because of the setting in advance feature.

In [4]:
rate_df=pd.DataFrame(index=[np.linspace(0.5,10,20)],columns=[np.linspace(0.5,10,20)])
for i in range(len(rate_df)):
    t=rate_df.columns[i][0]
    for j in range(i,len(rate_df)):
        tt=rate_df.index[j][0]
        delta_t=tt-t
        if i!=j:
            rate_df.iloc[j,i]=((1/sim_df.loc[tt,t].values[0])**(1/(2*delta_t))-1)*2
        else:
            rate_df.iloc[j,i]=np.nan
rate_df[0]=disc_func['par_rate'].to_list()
rate_df=rate_df.T.sort_index().T

for y in [2,3,5,10]:
    k=disc_func[disc_func['T']==2]['par_rate'].values[0]
    cap_v=0
    for i in range(1,2*y):
        cap_v+=max(rate_df.iloc[i,i]-k,0)/(rate_df.iloc[i,0]/2+1)**(2*rate_df.index[i][0])
    print(f'{y}-year caps=',cap_v)


2-year caps= 0.0199015047549638
3-year caps= 0.08354770524889271
5-year caps= 0.1764448460002749
10-year caps= 0.26391258914512183


6. Value a 5-year resettable cap with a strike of 0.07 and semiannual caplets. The cashflows from each caplet are 0.50max(0,L0.5−L0.0), 0.50max(0,L1.0−L0.50), etc. where Lt denotes the six-month rate at time t implied by the six-month zero-coupon bond price B(t, t + 0.50).

In [5]:
cap_v=0
for i in range(2*5):
    cap_v+=0.5*max(rate_df.iloc[i+1,i+1]-rate_df.iloc[i,i],0)/(rate_df.iloc[i,0]/2+1)**(2*rate_df.index[i][0])
print(cap_v)

0.022744843904414377


7. Value a 5-year CMS cap with a strike of 0.05 (semiannual caplets for simplicity). The cash flow for the caplet expiring at time t + 0.50 is 0.50max(0,CMSt − .05), where CMSt denotes the 5-year par rate implied by the vector of zero-coupon bond prices at time t. Note that the first caplet is omitted because of the setting in advance feature.

In [6]:
cap_v=0
for i in range(1,2*5):
    cap_v+=0.5*max(rate_df.iloc[i+10-1,i]-0.05,0)/(rate_df.iloc[i,0]/2+1)**(2*rate_df.index[i][0])
print(cap_v)

0.11858720272147864
